In [8]:
import seaborn as sns
from astropy.io import fits
from multiprocessing import Pool
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from scipy.stats import binned_statistic_2d
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.table import Table
from maps import data_loader
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.linewidth']= 3
plt.rcParams['xtick.major.size'] =8
plt.rcParams['ytick.major.size'] =8
plt.rcParams['xtick.minor.size'] =4
plt.rcParams['ytick.minor.size'] =4
plt.rcParams['xtick.major.width'] =5
plt.rcParams['ytick.major.width'] =5
plt.rcParams['xtick.minor.width'] =5
plt.rcParams['ytick.minor.width'] =5
plt.rcParams['font.size'] = 35
plt.rcParams['figure.figsize'] = (12,12)

In [3]:
def load_datasets(orig=True):

    if orig:
        TNG50 = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/TNG50_Rot_blobsLike_orig_0.03_0.055.csv')[['objid','likelihood','galsky','skysig','LLR']]
        TNG = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/TNG_Rot_blobsLike_orig_0.03_0.055.csv')
        Illustris = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/Illustris_Rot_blobsLike_orig_0.03_0.055.csv')[['objid','likelihood','LLR']]
        
        TNG50_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/df_TNG50_ordered_Mgt9.5_skysub_orig.csv',sep=' ')[['objid','mag','sersic_n_r','flag_r', 'sersic_rhalf_r',\
                                                                                                                    'StellarMasses_in_r30pkpc','SFR_MsunPerYrs_in_all_10Myrs']]
        TNG_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/df_TNG_ordered_Mgt9.5_skysub_orig.csv',sep=' ')[['objid','mag']]
        Illustris_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/df_ill_ordered_Mgt9.5_skysub_orig.csv',sep=' ')#[['objid','mag']]
        
        TNG50 = TNG50.merge(TNG50_mag, on='objid')
        TNG = TNG.merge(TNG_mag, on='objid')
        Illustris = Illustris.merge(Illustris_mag, on='objid')      
        
    else:
        TNG50 = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/TNG50_Rot_blobsLike_orig_0.03_0.055_magmatch.csv')
        TNG = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/TNG_Rot_blobsLike_orig_0.03_0.055_magmatch.csv')
        Illustris = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/Illustris_Rot_blobsLike_orig_0.03_0.055_magmatch.csv')
        
        TNG_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/df_TNG_ordered_Mgt9.5_skysub_magmatch.csv',sep=' ')[['objid','mag']]
        TNG50_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/df_TNG50_ordered_Mgt9.5_skysub_magmatch.csv',sep=' ')[['objid','mag']]
        TNG50 = TNG50.merge(TNG50_mag, on='objid')
        TNG = TNG.merge(TNG_mag, on='objid')      
        
    TNG50_mhalo = pd.read_csv('/scratch/lzanisi/pixel-cnn/TNG50_cutouts/TNG50_mhalo.csv')
    TNG50 = TNG50.merge(TNG50_mhalo, on='objid')
    TNG50 = TNG50.rename(columns={'galsky':'sky [nmaggie]','flag_r':'flag_sersic','mag_x':'mag','StellarMasses_in_r30pkpc':'Mstar', 'SFR_MsunPerYrs_in_all_10Myrs':'SFR'})

    SDSS = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/cleaned_df_bis_0.03_0.055_blobsLike_skysub.dat',sep=' ')[35000:]#[['objid','galcount','','LCentSat','mag']]
    SDSS_all = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/DataFrames_LLR/SDSS_Rot_blobsLike_0.03_0.055_all.csv')[['galcount','likelihood','LLR']]
    #SDSS_mag = pd.read_csv('/scratch/lzanisi/pixel-cnn/data/cleaned_df_bis_0.03_0.055_blobsLike_skysub_onlymag.dat', sep= ' ')
    
    SDSS = SDSS.merge(SDSS_all, on='galcount')
    #SDSS = SDSS.query('MhaloL>0').sample(frac=9196/len(SDSS_all))
    #SDSS = SDSS_L.merge(SDSS, on='galcount')
    
    # add physical properties to TNG
    df_f = pd.read_csv('/scratch/lzanisi/pixel-cnn/TNG_cutouts/snap_95_unfiltered_bis.csv')[['Unnamed: 0','SFR']]
    TNG = pd.merge(TNG, df_f, left_on='Illustris_ID_2_2', right_on='Unnamed: 0')
    mhalo = pd.read_csv('/scratch/lzanisi/pixel-cnn/TNG_cutouts/snap_95_mhalomean.csv')[['Unnamed: 0', 'ParentDM','M_BH']]
    TNG = pd.merge(TNG, mhalo, left_on='Illustris_ID_2_2', right_on='Unnamed: 0')
    centrals_jstar = pd.read_csv('/scratch/lzanisi/pixel-cnn/TNG_cutouts/TNG_jstar_s95_lzanisi.csv') #this contains only centrals
    TNG = pd.merge(TNG, centrals_jstar, left_on = 'Illustris_ID_2_2', right_on='Illustris_ID_2_1', how='outer' )
    TNG['LCentSat'] = TNG['sJ_star_1re'].apply(lambda x: 0 if x!=x else 1)
    TNG['SFR'] = TNG['SFR'].apply(lambda x: np.log10(x) if x==x else np.nan)
    TNG50['SFR'] = TNG50['SFR'].apply(lambda x: np.log10(x) if x==x else np.nan)

    #clean
    TNG = TNG.query('likelihood>3000 & sersic_n>0 & LLR>-50 & sersic_n<7  & LLR<500')
    TNG50 = TNG50.query('likelihood>3000 & sersic_n_r>0 & LLR>-50 & sersic_n_r<7  & LLR<500')

    Illustris = Illustris.query('likelihood>3000 & sersic_n>0 & LLR>-50  &  sersic_n<7 & LLR<500')
    SDSS = SDSS.query('likelihood>3000  & LLR>-50  & LLR<500 & GalSky_err>0 ')
    
    #rename
    SDSS['sky [nmaggie]'] = SDSS['GalSky'].apply(lambda x: 10**(-0.4*(x-22.5)  )*0.396**2)
    Illustris = Illustris.rename(columns={'galsky':'sky [nmaggie]','sersic_n':'n_bulge'})
    TNG = TNG.rename(columns={'sky':'sky [nmaggie]','sersic_n':'n_bulge','mag_x':'mag','LogMass30':'Mstar','ParentDM':'Mhalo'}) 
    TNG50 = TNG50.rename(columns={'galsky':'sky [nmaggie]','sersic_n_r':'n_bulge','mag_x':'mag','StellarMasses_in_r30pkpc':'Mstar', 'SFR_MsunPerYrs_in_all_10Myrs':'SFR'})
    SDSS = SDSS.rename(columns={'MsMendSerExp':'Mstar','MhaloL':'Mhalo'})
    
    
    TNG['$logR_e \ [arcsec]$'] = TNG['sersic_rhalf'].apply(lambda x: np.log10(0.396*x))
    TNG['$n_{ser}$'] = TNG['n_bulge'].copy()
    TNG['sSFR'] = TNG.apply(lambda row: row.SFR-row.Mstar,axis=1)
    TNG['sSFR'] = TNG['sSFR'].apply(lambda x: -12.5 if x<-12.5 or x!=x else x)
    
    
    TNG50['$logR_e \ [arcsec]$'] = TNG50['sersic_rhalf_r'].apply(lambda x: np.log10(0.396*x))
    TNG50['$n_{ser}$'] = TNG50['n_bulge'].copy()
    TNG50['sSFR'] = TNG50.apply(lambda row: row.SFR-row.Mstar,axis=1)
    TNG50['sSFR'] = TNG50['sSFR'].apply(lambda x: -12.5 if x<-12.5 or x!=x else x)
    TNG50['Mstar'] = TNG50['Mstar'].apply(lambda x: x +np.log10(0.68))
    
    Illustris['$logR_e \ [arcsec]$'] = Illustris['sersic_rhalf'].apply(lambda x: np.log10(0.396*x))
    Illustris['$n_{ser}$'] = Illustris['n_bulge'].copy()
    Illustris['sSFR'] = Illustris.apply(lambda row: row.SFR-row.Mstar,axis=1)
    Illustris['sSFR'] = Illustris['sSFR'].apply(lambda x: -12.5 if x<-12.5 or x!=x else x)
    
    SDSS['$logR_e \ [arcsec]$'] = SDSS['r_bulge'].apply(np.log10)
    SDSS = SDSS[SDSS['$logR_e \ [arcsec]$']>-1]
    SDSS['$n_{ser}$'] = SDSS['n_bulge'].copy()
    SDSS['sSFR'] = SDSS.apply(lambda row: row.SFR-row.Mstar,axis=1)
    SDSS['sSFR'] = SDSS['sSFR'].apply(lambda x: -12.5 if x<-12.5 or x!=x else x)
    SDSS['Mhalo'] = SDSS['Mhalo'].apply(lambda x: 0 if x==0 else x + np.log10(1.1))
    
    
    return SDSS, TNG50, TNG, Illustris

In [10]:
def loader(dataset,model1, model2):
    print('loading {} ...'.format(dataset))
    
    loader = data_loader(dataset=dataset, model=model1,model2=model2)
    LLR, ids = loader.load(map_type='LLR')
    data, _ = loader.load(map_type='data_rot')
     
    return LLR, data, ids

In [4]:
SDSS, TNG50, TNG, Illustris = load_datasets(orig=True)

/scratch/lzanisi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
SDSS_cen = SDSS.query('LCentSat==1')
TNG_cen = TNG.query('LCentSat==1')
TNG50_cen = TNG50.query('LCentSat==1')
Illustris_cen = Illustris.query('LCentSat==1')

SDSS_sat = SDSS.query('LCentSat==2')
TNG_sat = TNG.query('LCentSat==0')
TNG50_sat = TNG50.query('LCentSat==0')
Illustris_sat = Illustris.query('LCentSat==0')

In [6]:
SDSS_Q = SDSS.query('sSFR<-11')
SDSS_SF = SDSS.query('sSFR>-11')

TNG_Q = TNG.query('sSFR<-11')
TNG_SF = TNG.query('sSFR>-11')

TNG50_Q = TNG50.query('sSFR<-11')
TNG50_SF = TNG50.query('sSFR>-11')

Illustris_Q = Illustris.query('sSFR<-11')
Illustris_SF = Illustris.query('sSFR>-11')


In [7]:
SDSS_cen_Q = SDSS.query('LCentSat==1 & sSFR<-11')
TNG_cen_Q = TNG.query('LCentSat==1 & sSFR<-11')
TNG50_cen_Q = TNG50.query('LCentSat==1 & sSFR<-11')
Illustris_cen_Q = Illustris.query('LCentSat==1 & sSFR<-11')

SDSS_cen_SF = SDSS.query('LCentSat==1 & sSFR>-11')
TNG_cen_SF = TNG.query('LCentSat==1 & sSFR>-11')
TNG50_cen_SF = TNG50.query('LCentSat==1 & sSFR>-11')
Illustris_cen_SF = Illustris.query('LCentSat==1 & sSFR>-11')

In [9]:
model1 = '/scratch/lzanisi/pixel-cnn/trained/0.03_0.055/asinh_SDSS_blobsLike_0.03_0.055_old/1Msteps/pixelcnn_out'
model2 ='/scratch/lzanisi/pixel-cnn/trained/0.03_0.055/asinh_NewSersicBlobs_SerOnly_0.03_0.055_old/1Msteps/pixelcnn_out'

In [13]:
LLR_TNG50, data_TNG50, ids_TNG50 = loader(dataset='TNG50', model1=model1, model2=model2)
LLR_TNG, data_TNG, ids_TNG = loader(dataset='TNG', model1=model1, model2=model2)
LLR_SDSS, data_SDSS, ids_SDSS = loader(dataset='SDSS', model1=model1, model2=model2)

loading TNG50 ...


FileNotFoundError: [Errno 2] No such file or directory: '/scratch/lzanisi/pixel-cnn/outputs/TNG50/lmap_asinh_SDSS_blobsLike_0.03_0.055_old_1Msteps_test_orig.pkl'